# Before you start :
   - These exercises are related to the Pivot table and correlation lessons.
   - Keep in mind that you need to use some of the functions you learned in the previous lessons.
   - All datasets are provided in the `your-code` folder of this lab.
   - Elaborate your codes and outputs as much as you can.
   - Try your best to answer the questions and complete the tasks and most importantly enjoy the process!!!

#### Import all the libraries that are necessary.

In [1]:
import pandas as pd
import numpy as np


#### Open the ` Employee.csv` file and name your dataset `employee`.

In [9]:
employee = pd.read_csv(r"data\Employee.csv")

In [12]:
employee

,Name,Department,Education,Gender,Title,Years,Salary
0,Jose,IT,Bachelor,M,analyst,1,35
1,Maria,IT,Master,F,analyst,2,30
2,David,HR,Master,M,analyst,2,30
3,Sonia,HR,Bachelor,F,analyst,4,35
4,Samuel,Sales,Master,M,associate,3,55
5,Eva,Sales,Bachelor,F,associate,2,55
6,Carlos,IT,Master,M,VP,8,70
7,Pedro,IT,Phd,M,associate,7,60
8,Ana,HR,Master,F,VP,8,70


#### What's the mean salary by department?
Hint : There are 2 ways to do it - one way is with groupby and the other way is with pivot tables. Do it both ways. 


In [14]:
# 1. Group By

employee.groupby(['Department']).agg({'Salary':'mean'}).reset_index()

,Department,Salary
0,HR,45.00
1,IT,48.75
2,Sales,55.00


In [15]:
# 2. Pivot Table
# Since the Pivot Table gives you the mean of the selected value I do not have to specify anything.

employee.pivot_table(index= ['Department'], values = ['Salary'])

,Salary
Department,
HR,45.00
IT,48.75
Sales,55.00


#### We want now to group the data by more than one column. Use Pandas pivot table function to select the mean salary by department and title.

In [17]:
employee.pivot_table(index= ['Department', 'Title'], values = ['Salary']).reset_index()


,Department,Title,Salary
0,HR,VP,70.0
1,HR,analyst,32.5
2,IT,VP,70.0
3,IT,analyst,32.5
4,IT,associate,60.0
5,Sales,associate,55.0


#### We want to display the number of years of work experience and the salary for each Title. Add Years as a column in your pivot table.
*Hint: Use Pandas doc(see References in README.md) to explore the pivot_table() arguments*

In [22]:
employee.pivot_table(index='Title', values=['Years', 'Salary'],aggfunc={'Years': 'mean', 'Salary': 'mean'})


,Salary,Years
Title,,
VP,70.000000,8.00
analyst,32.500000,2.25
associate,56.666667,4.00


#### We want to know the information about the mean salary and number of employees in each department and for each job title.

In [24]:
df = employee.pivot_table(index= ['Department', 'Title'], values = ['Salary', 'Name'],aggfunc={'Salary': 'mean', 'Name': pd.Series.nunique}).reset_index()
df.columns = ['Department','Title', 'Num_Employees', 'MeanSalary']
df

,Department,Title,Num_Employees,MeanSalary
0,HR,VP,1,70.0
1,HR,analyst,2,32.5
2,IT,VP,1,70.0
3,IT,analyst,2,32.5
4,IT,associate,1,60.0
5,Sales,associate,2,55.0


#### We want to know for each department and for each job title the Salary's median, minimum, maximum and standard deviation. Comment your results.

In [30]:
df = employee.groupby(['Department', 'Title'])['Salary'].agg(['median', 'min', 'max', 'std']).reset_index()
df.columns = ['Department','Title', 'MedianSalary', 'MinSalary', 'MaxSalary', 'StdSalary']
df


,Department,Title,MedianSalary,MinSalary,MaxSalary,StdSalary
0,HR,VP,70.0,70,70,NaN
1,HR,analyst,32.5,30,35,3.535534
2,IT,VP,70.0,70,70,NaN
3,IT,analyst,32.5,30,35,3.535534
4,IT,associate,60.0,60,60,NaN
5,Sales,associate,55.0,55,55,0.000000


In [ ]:
# The Std Salary has some NaN values because in these cases there is just one employee with the title in the department and the std can not be calculated with just one value.
# Therefore I would suggest to fill the NaN values with 0

#### Based on your comments, fill the missing values with an appropriate value.

In [32]:
df['StdSalary'] = df['StdSalary'].fillna(0).round(2)
df

,Department,Title,MedianSalary,MinSalary,MaxSalary,StdSalary
0,HR,VP,70.0,70,70,0.00
1,HR,analyst,32.5,30,35,3.54
2,IT,VP,70.0,70,70,0.00
3,IT,analyst,32.5,30,35,3.54
4,IT,associate,60.0,60,60,0.00
5,Sales,associate,55.0,55,55,0.00


#### The stake holders want to know for each department the number of employees and how much money is spend on salaries. Could you provide that information? 

In [33]:
df = employee.pivot_table(index= ['Department'], values = ['Name', 'Salary'],aggfunc={'Name': 'count', 'Salary': 'sum', }).reset_index()
df.columns = ['Department', 'Num_Employees', 'Total_Salary']
df


,Department,Num_Employees,Total_Salary
0,HR,3,135
1,IT,4,195
2,Sales,2,110


#### For each Department and Title, we want to know the the total years of work experience and the mean salary.

In [37]:
employee.pivot_table(index=['Department', 'Title'], values=['Years', 'Salary'],aggfunc={'Years': 'sum', 'Salary': 'mean'}).reset_index()


,Department,Title,Salary,Years
0,HR,VP,70.0,8
1,HR,analyst,32.5,6
2,IT,VP,70.0,8
3,IT,analyst,32.5,3
4,IT,associate,60.0,7
5,Sales,associate,55.0,5


### Bonus

#### We now want to compute the mean salary after removing the maxima for each Department. Create the appropriate pivot table.

Hint: Write a custom function.

In [77]:
def drop_max_salary(df):
    # Get the maximum salary for each department
    # https://stackoverflow.com/questions/51495782/return-all-rows-after-groupby-pandas-i-e-not-a-reduced-number-of-rows-that-is
    max_salaries = df.groupby(['Department'])['Salary'].transform('max')
    # Get the rows that are lower than the maximum salary
    df_filtered = df[df['Salary'] < max_salaries]
    return df_filtered

# Remove the maximum salaries for each department
employee_filtered = drop_max_salary(employee)

# Calculate the mean salary for each department after removing the maxima
employee_filtered.pivot_table(index=['Department'], values=['Salary'], aggfunc='mean')


,Salary
Department,
HR,32.500000
IT,41.666667


In [ ]:
# We just get values for HR and IT Department because the Sales Department just has one employee and we dropped it when we performed the function.